# Reserva de Hotel con Middleware para Miembros Prioritarios

Este cuaderno demuestra el uso de **middleware basado en funciones** utilizando el Microsoft Agent Framework. Partimos del ejemplo de flujo de trabajo condicional y añadimos una capa de middleware que otorga privilegios especiales a los miembros prioritarios.

## Lo que aprenderás:
1. **Middleware basado en funciones**: Interceptar y modificar resultados de funciones
2. **Acceso al contexto**: Leer y modificar `context.result` después de la ejecución
3. **Implementación de lógica empresarial**: Beneficios para miembros prioritarios
4. **Sobrescribir resultados**: Cambiar los resultados de funciones según el estado del usuario
5. **Mismo flujo de trabajo, diferentes resultados**: Cambios de comportamiento impulsados por middleware

## Arquitectura del flujo de trabajo con Middleware:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Diferencia clave con el flujo de trabajo condicional:

**Sin Middleware** (14-conditional-workflow.ipynb):
- París no tiene habitaciones → Redirigir a alternative_agent

**Con Middleware** (este cuaderno):
- Usuario regular + París → No hay habitaciones → Redirigir a alternative_agent
- Usuario prioritario + París → 🌟 ¡Sobrescritura del middleware! → Disponible → Redirigir a booking_agent

## Requisitos previos:
- Microsoft Agent Framework instalado
- Comprensión de flujos de trabajo condicionales (ver 14-conditional-workflow.ipynb)
- Token de GitHub o clave de API de OpenAI
- Comprensión básica de patrones de middleware


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Paso 1: Definir modelos Pydantic para salidas estructuradas

Estos modelos definen el **esquema** que los agentes devolverán. Hemos añadido un campo `priority_override` para registrar cuando el middleware modifica el resultado de disponibilidad.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Paso 2: Definir la Base de Datos de Miembros Prioritarios

Para esta demostración, simularemos una base de datos de membresía prioritaria. En producción, esto consultaría una base de datos real o una API.

**Miembros Prioritarios:**
- `alice@example.com` - Miembro VIP
- `bob@example.com` - Miembro Premium  
- `priority_user` - Cuenta de prueba


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Paso 3: Crear la Herramienta de Reservas de Hotel

Igual que el flujo de trabajo condicional, ¡pero ahora será interceptado por middleware!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Paso 4: 🌟 Crear Middleware de Verificación de Prioridad (¡LA FUNCIÓN CLAVE!)

Esta es la **funcionalidad principal** de este notebook. El middleware:

1. **Intercepta** la llamada a la función hotel_booking
2. **Ejecuta** la función normalmente llamando a `next(context)`
3. **Inspecciona** el resultado en `context.result`
4. **Sobrescribe** el resultado si el usuario es prioritario y no hay habitaciones disponibles
5. **Devuelve** el resultado modificado al agente

**Patrón Clave:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Paso 5: Definir funciones de condición para el enrutamiento

Las mismas funciones de condición que en el flujo de trabajo condicional: inspeccionan la salida estructurada para determinar el enrutamiento.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Paso 6: Crear un Ejecutor de Visualización Personalizado

El mismo ejecutor que antes: muestra el resultado final del flujo de trabajo.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Paso 7: Cargar Variables de Entorno

Configura el cliente LLM (GitHub Models o OpenAI).


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Paso 8: Crear agentes de IA con middleware

**DIFERENCIA CLAVE:** ¡Al crear el availability_agent, pasamos el parámetro `middleware`!

Así es como inyectamos el priority_check_middleware en la canalización de invocación de funciones del agente.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Paso 9: Construir el flujo de trabajo

La misma estructura de flujo de trabajo que antes: enrutamiento condicional basado en la disponibilidad.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Paso 10: Caso de Prueba 1 - Usuario Regular en París (Sin Sobrescritura)

Un usuario regular intenta reservar París → No hay habitaciones → Redirige a alternative_agent


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Paso 11: Caso de Prueba 2 - 🌟 Usuario Prioritario en París (¡CON Sobrescritura!)

Un miembro prioritario intenta reservar en París → No hay habitaciones inicialmente → 🌟 ¡El middleware sobrescribe! → Redirige al booking_agent

**¡Esta es la demostración clave del poder del middleware!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Paso 12: Caso de Prueba 3 - Usuario Prioritario en Estocolmo (Ya Disponible)

El usuario prioritario intenta Estocolmo → Habitaciones disponibles → No se necesita anulación → Se dirige a booking_agent

¡Esto demuestra que el middleware solo actúa cuando es necesario!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Puntos Clave y Conceptos de Middleware

### ✅ Lo que has aprendido:

#### **1. Patrón de Middleware Basado en Funciones**

El middleware intercepta llamadas a funciones utilizando una función asíncrona simple:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. Acceso al Contexto y Sobrescritura de Resultados**

- `context.function` - Accede a la función que se está llamando
- `context.arguments` - Lee los argumentos de la función
- `context.kwargs` - Accede a parámetros adicionales
- `await next(context)` - Ejecuta la función
- `context.result` - Lee/modifica el resultado de la función

#### **3. Implementación de Lógica de Negocio**

Nuestro middleware implementa beneficios para miembros prioritarios:
- **Usuarios regulares**: Sin modificaciones, flujo estándar
- **Usuarios prioritarios**: Sobrescribe "sin disponibilidad" → "disponible"
- **Lógica condicional**: Solo sobrescribe cuando es necesario

#### **4. Mismo Flujo de Trabajo, Resultados Diferentes**

El poder del middleware:
- ✅ Sin cambios en la estructura del flujo de trabajo
- ✅ Sin cambios en la función de la herramienta
- ✅ Sin cambios en la lógica de enrutamiento condicional
- ✅ Solo middleware → ¡Comportamiento diferente!

### 🚀 Aplicaciones en el Mundo Real:

1. **Funciones VIP/Premium**
   - Sobrescribir límites de tasa para usuarios premium
   - Proporcionar acceso prioritario a recursos
   - Desbloquear funciones premium dinámicamente

2. **Pruebas A/B**
   - Dirigir usuarios a diferentes implementaciones
   - Probar nuevas funciones con usuarios específicos
   - Implementaciones graduales de funciones

3. **Seguridad y Cumplimiento**
   - Auditar llamadas a funciones
   - Bloquear operaciones sensibles
   - Hacer cumplir reglas de negocio

4. **Optimización de Rendimiento**
   - Cachear resultados para usuarios específicos
   - Omitir operaciones costosas cuando sea posible
   - Asignación dinámica de recursos

5. **Manejo de Errores y Reintentos**
   - Capturar y manejar errores de manera elegante
   - Implementar lógica de reintento
   - Alternar a implementaciones alternativas

6. **Registro y Monitoreo**
   - Rastrear tiempos de ejecución de funciones
   - Registrar parámetros y resultados
   - Monitorear patrones de uso

### 🔑 Diferencias Clave con los Decoradores:

| Característica | Decorador | Middleware |
|----------------|-----------|------------|
| **Alcance** | Función única | Todas las funciones en el agente |
| **Flexibilidad** | Fijo en la definición | Dinámico en tiempo de ejecución |
| **Contexto** | Limitado | Contexto completo del agente |
| **Composición** | Múltiples decoradores | Cadena de middleware |
| **Consciente del Agente** | No | Sí (acceso al estado del agente) |

### 📚 Cuándo Usar Middleware:

✅ **Usa middleware cuando:**
- Necesitas modificar el comportamiento basado en el estado del usuario/sesión
- Quieres aplicar lógica a múltiples funciones
- Necesitas acceso al contexto a nivel de agente
- Estás implementando preocupaciones transversales (registro, autenticación, etc.)

❌ **No uses middleware cuando:**
- Validación simple de entradas (usa Pydantic)
- Lógica específica de una función (mantenla en la función)
- Modificaciones puntuales (simplemente cambia la función)

### 🎓 Patrones Avanzados:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 Conceptos Relacionados:

- **Middleware de Agente**: Intercepta llamadas a agent.run()
- **Middleware de Función**: Intercepta llamadas a funciones de herramientas (¡lo que usamos!)
- **Cadena de Middleware**: Secuencia de middleware que se ejecuta en orden
- **Propagación de Contexto**: Pasa el estado a través de la cadena de middleware



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
